In [ ]:
import pickle as pkl
import json

In [ ]:
with open("dev_labeled_datas.pkl","rb") as fi_dev_labeled:
    dev_labels_data = pkl.load(fi_dev_labeled)
    
with open("test_labeled_datas.pkl","rb") as fi_test_labeled:
    test_labels_data = pkl.load(fi_test_labeled)

gpt_label_data = dev_labels_data+test_labels_data

In [ ]:
label_type_list = ['root', ' sasubj-obj', ' object', ' sasubj', ' dfsubj', ' subj', ' subj-in', ' obj', ' att', ' adv', ' cmp', ' coo', ' pobj', ' iobj', ' de', ' adjct', ' app', ' exp', ' punc', ' frag', ' repet', ' bckg', ' cause', ' comp', ' cond', ' cont', ' elbr', ' enbm', ' eval', ' expl', ' joint', ' manner', ' rstm', ' temp', ' tp-chg', ' prob-sol', ' qst-ans', ' stm-rsp', ' req-proc']
label_type_list = [_.strip() for _ in label_type_list]
# (会_0_0,开车_0_1,root) (tail,head,dep)
with open("aug_train_data.conll","w",encoding="utf8") as fo_aug_train:
    for once_labeled_data in gpt_label_data:
        once_labeled = once_labeled_data[3]
        once_labeled_list = once_labeled.split("<>")
        # 构造依赖关系的字典
        turn_pos_idx2head = {}
        for once_labeled_list_item in once_labeled_list:
            try:
                once_labeled_list_item = list(once_labeled_list_item.strip("(").strip(")").split(","))
                once_labeled_list_item = [_.strip() for _ in once_labeled_list_item]
                tail_info = once_labeled_list_item[0]
                tail_info = tail_info.split("_")
                
                head_info = once_labeled_list_item[1]
                head_info = head_info.split("_")
                
                turn_pos_idx2head.setdefault(int(tail_info[1]),{})
                turn_pos_idx2head[int(tail_info[1])][int(tail_info[2])] = [head_info[2],once_labeled_list_item[2]]
            except:
                continue
        
        dialog_text = once_labeled_data[1]
        formatted_sentences = []
        for idx,once_turn in enumerate(dialog_text):
            for jdx,once_words in enumerate(once_turn.split()):
                if idx in turn_pos_idx2head and jdx in turn_pos_idx2head[idx] and turn_pos_idx2head[idx][jdx][1] in label_type_list:
                    formatted_sentences.append(f"{jdx+1}\t{once_words}\t_\t_\t_\t_\t{turn_pos_idx2head[idx][jdx][0]}\t{turn_pos_idx2head[idx][jdx][1]}\t_\t_")
                else:
                    formatted_sentences.append(f"{jdx+1}\t{once_words}\t_\t_\t_\t_\t_\t_\t_\t_")
                    
            formatted_sentences.append("")
                    
        # print('\n'.join(formatted_sentences))
        fo_aug_train.write('\n'.join(formatted_sentences)+"\n")
    